In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# from AlgoTestFactory import *

# altf1 = AlgoTestFactory([0, 10000], 15, savePath="Combined_4_test2", storeAccuracies=True)

# altf1.run()

In [3]:
from SktWsegRWR_utf8 import *
import pickle
import ProbData
from ProbModels import *
import multiprocessing
import math
import json

Dataloader Started[Prob]...
Dataloader Finished[Prob]...


In [5]:
pb = ProbModels(fullCo_oc_mat = ProbData.fullCo_oc_mat, unigram_counts = ProbData.unigram_counts,
               cng2cngFullMat = ProbData.cng2cngFullMat, cng2index_dict = ProbData.cng2index_dict,
               w2w_samecng_fullmat=ProbData.w2w_samecng_fullmat, samecng_unigram_counts=ProbData.samecng_unigram_counts,
               v2c_fullMat = ProbData.v2c_fullMat)

goodFileDict = pickle.load(open('mergedGood_v2.p', 'rb'))
algo = SktWsegRWR(
            w2w_modelFunc = pb.get_w2w_mat, 
            t2t_modelFunc = pb.get_cng2cng_mat,
            v2c_modelFunc = pb.get_v2c_ranking,
            sameCng_modelFunc = pb.get_w2w_samecng_mat,
            partition=[0.2, 0.5, 0.2, 0.1]
        )

In [6]:
def loadSentence(fName, folderTag):
        # print('File: ', fName)
        try:
            dcsObj = pickleFixLoad('../Text Segmentation/DCS_pick/' + fName)           
            if folderTag == "C1020" :
                sentenceObj = pickleFixLoad('../TextSegmentation/corrected_10to20/' + fName)
            else:
                sentenceObj = pickleFixLoad('../TextSegmentation/Pickle_Files/' + fName)

        except (KeyError, EOFError, pickle.UnpicklingError) as e:
            return None, None
        return(sentenceObj, dcsObj)
    
def Accuracy(prediction, dcsObj):
    solution = [rom_slp(c) for c in dcsObj.dcs_chunks]
    ac = 100*sum(list(map(lambda x: x in solution, prediction)))/len(prediction)
    return ac

In [9]:
for f in list(goodFileDict.keys())[0:1]:
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result = algo.predict(sentenceObj, dcsObj)
        print(result)
        SeeSentence(sentenceObj)

None
praBAtakAle samprApte kANkzitavye samAgame   
Analyzing  praBAtakAle
0 :  praBA ['praBA_2'] [{'noun': ['nom. sg. f.']}]
0 :  pra ['pra'] []
0 :  pra ['pra'] []
8 :  ale ['ala'] [{'noun': ['acc. du. n.', 'nom. du. n.', 'loc. sg. n.']}]
8 :  ale ['ala', 'al'] [{'noun': ['voc. du. n.']}, {'noun': ['voc. sg. m.']}]
3 :  BAta ['BA_1'] [{'verb': ['imp. [2] ac. pl. 2']}]
3 :  BAta ['BAta', 'BA_1'] [{'compound': ['iic.']}, {'verb': ['pp.']}]
4 :  ata ['at'] [{'verb': ['imp. [1] ac. sg. 2']}]
4 :  Ata ['at'] [{'verb': ['pft. ac. pl. 2', 'pft. ac. sg. 3', 'pft. ac. sg. 1']}]
7 :  kAle ['kAla_1', 'kAla_2'] [{'noun': ['loc. sg. m.']}, {'noun': ['loc. sg. m.', 'acc. du. n.', 'nom. du. n.', 'loc. sg. n.']}]
7 :  kAle ['kAla_2'] [{'noun': ['voc. du. n.']}]
7 :  kAle ['kAla_1'] [{'noun': ['acc. du. n.', 'nom. du. n.', 'loc. sg. n.', 'acc. du. f.', 'nom. du. f.']}]
7 :  ka ['ka_1'] [{'compound': ['iic.']}]
7 :  kA ['ka_1'] [{'noun': ['nom. sg. f.']}]
Analyzing  samprApte
0 :  samprApte ['samprApt'